# Descriptive statistics

In [ ]:
import pandas as pd

def calculate_statistics(df, output_file):
    # Create an ExcelWriter object for output (engine='xlsxwriter')
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        # Compute grouped statistics by Length and save to a separate sheet per variable
        for column in ['GPT_FAM_probs']:
            stats_by_length = df.groupby('Length').agg({
                column: ['mean', 'std', 'min', 'max', 'count']
            })
        
            stats_by_length.columns = ['_'.join(col).strip() for col in stats_by_length.columns.values]
            stats_by_length.to_excel(writer, sheet_name=column)
        
        print(f"统计量已保存为 {output_file}")

# Load input Excel file
df = pd.read_excel('27624_word_7_cleaned.xlsx')

# Run and save statistics to output Excel file
calculate_statistics(df, 'unique_counts_statistics_word_GPTFAM.xlsx')

In [ ]:
# Paired-samples t-test
import pandas as pd
from scipy import stats
import numpy as np

# file paths for word and expression datasets
word_file_path = '27624_word_7_cleaned.xlsx'
expression_file_path = '27624_expression_7_cleaned.xlsx' 

# Load Excel files
word_df = pd.read_excel(word_file_path)
expression_df = pd.read_excel(expression_file_path)

# Select 'WORD', 'Length' and 'GPT_FAM_probs' from word; rename expression GPT_FAM_probs column
word_df_selected = word_df[['WORD', 'Length', 'GPT_FAM_probs']]
expression_df_selected = expression_df[['WORD', 'GPT_FAM_probs']].rename(columns={'GPT_FAM_probs': 'GPT_FAM_probs_expression'})

# Merge on 'WORD' to keep shared items
merged_df = pd.merge(word_df_selected, expression_df_selected, on='WORD')

# Get unique Length values
length_values = merged_df['Length'].unique()

# For each Length group, perform paired t-test between GPT_FAM_probs and GPT_FAM_probs_expression
for length in length_values:
    group = merged_df[merged_df['Length'] == length]
    
    # sample size for the current group
    sample_size = len(group)
    
    # compute means and standard deviations
    mean_word = group['GPT_FAM_probs'].mean()
    mean_expression = group['GPT_FAM_probs_expression'].mean()
    std_word = group['GPT_FAM_probs'].std()
    std_expression = group['GPT_FAM_probs_expression'].std()

    # paired t-test
    t_stat, p_value = stats.ttest_rel(group['GPT_FAM_probs'], group['GPT_FAM_probs_expression'])
    
    # degrees of freedom
    df = len(group['GPT_FAM_probs']) - 1
    
    # Cohen's d for paired samples
    diff = group['GPT_FAM_probs'] - group['GPT_FAM_probs_expression']
    cohen_d = diff.mean() / diff.std(ddof=1)

    # print formatted results
    print(f"Length = {length} 组内的样本量: {sample_size}")
    print(f"t 统计量: {t_stat:.4f}, 自由度 (df): {df}, p 值: {p_value:.4f}")
    print(f"均值 (word): {mean_word:.4f}, 标准差 (word): {std_word:.4f}")
    print(f"均值 (expression): {mean_expression:.4f}, 标准差 (expression): {std_expression:.4f}")
    print(f"Cohen's d (效应量): {cohen_d:.4f}")
    print("-" * 50)


# Correlation

In [3]:
# Compute bivariate correlations by word length - single-character items
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

# Load dataset
data = pd.read_excel('27624_expression_7_cleaned.xlsx')  

# Group by Length and compute Pearson correlation per group
results = []
for name, group in data.groupby('Length'):
    # Drop rows with NaN or infinite values in the relevant columns
    group = group.replace([np.inf, -np.inf], np.nan).dropna(subset=['GPT_FAM_probs','Human_FAM_Liu']) # 'SUBTLEX_logWF','SUBTLEX_logW_CD','SUBTLEX_logCHR','SUBTLEX_logCHR_CD'(当算和词频的相关的时候)
    
    # Compute sample size
    sample_size = group.shape[0]

    # Require at least 2 observations to compute correlation
    if sample_size > 1:
        # Compute Pearson r and p-value, format to 4 decimals
        r, p_value = pearsonr(group['GPT_FAM_probs'], group['Human_FAM_Liu'])
        results.append((name, sample_size, f"{r:.4f}", f"{p_value:.4f}"))
    else:
        # Not enough data for this length
        results.append((name, sample_size, "N/A", "N/A"))

# Convert results to DataFrame for display
result_df = pd.DataFrame(results, columns=['WordLength', 'SampleSize', 'Pearsonr', 'PValue'])

# Print results
print(result_df)

   WordLength  SampleSize Pearsonr  PValue
0           1        2336   0.6161  0.0000
1           2           0      N/A     N/A
2           3           0      N/A     N/A
3           4           0      N/A     N/A


In [ ]:
# Compute bivariate correlations by word length - multi-character items
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

# Load data
data = pd.read_excel('27624_word_7_cleaned.xlsx')  

# Group by Length and compute correlation per group
results = []
for name, group in data.groupby('Length'):
    # Remove rows with NaN or infinite values (keep relevant English tokens)
    group = group.replace([np.inf, -np.inf], np.nan).dropna(subset=['GPT_FAM_probs', 'Human_FAM_M_Su']) # 'SUBTLEX_logWF','SUBTLEX_logW_CD'
    
    # Compute sample size
    sample_size = group.shape[0]

    # Require at least 2 observations to compute Pearson r
    if sample_size > 1:
        r, p_value = pearsonr(group['GPT_FAM_probs'], group['Human_FAM_M_Su'])
        # Format r and p-value to 3 decimals
        results.append((name, sample_size, f"{r:.3f}", f"{p_value:.3f}"))
    else:
        # Not enough data for this length
        results.append((name, sample_size, "N/A", "N/A"))

# Convert results to DataFrame for display
result_df = pd.DataFrame(results, columns=['WordLength', 'SampleSize', 'Pearsonr', 'PValue'])

# Print results
print(result_df)

In [ ]:
# Compute partial correlation controlling for Length
import pandas as pd
import pingouin as pg

# Load Excel file into a DataFrame
file_path = '27624_word_7_cleaned.xlsx' 
df = pd.read_excel(file_path)

# Drop rows missing the variables needed for the analysis
df_cleaned = df.dropna(subset=['GPT_FAM_probs', 'Human_FAM_M_Su', 'Length'])

# Compute partial correlation controlling for Length
partial_corr_result = pg.partial_corr(data=df_cleaned, x='GPT_FAM_probs', y='Human_FAM_M_Su', covar='Length')

# Print the partial correlation result; format p-value to 4 decimal places
print(partial_corr_result.to_string(formatters={'p-val': '{:.4f}'.format}))

# Regression

## Univariate Linear Regression

In [7]:
# Univariate linear regression - single-character items - LDT task
import pandas as pd
import statsmodels.formula.api as smf

# Load data file and specify columns to read
file_path = '27624_expression_7_cleaned_filtered.xlsx'  # Excel file path
y_column = 'zRT_LDT'  # dependent variable
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_Liu","SUBTLEX_logWF","SUBTLEX_logW_CD","SUBTLEX_logCHR","SUBTLEX_logCHR_CD"]  # predictors
length_column = 'Length'  # word length column

# Read Excel and select needed columns
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Remove rows with missing values in any predictor or the outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Collect regression results
final_results = []

# Loop over each Length group and run univariate OLS for each predictor
lengths = data_cleaned[length_column].unique()
for length in lengths:
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    for x in x_columns:
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)
        })

# Convert results list to DataFrame and display
final_results_df = pd.DataFrame(final_results)
print(final_results_df)

   Length           Variable  R-squared  Coefficient  P-value  Standard Error  \
0       1      GPT_FAM_probs      0.431       -0.138      0.0           0.008   
1       1   qwen_FAM_mean_30      0.394       -0.187      0.0           0.011   
2       1      Human_FAM_Liu      0.264       -0.216      0.0           0.017   
3       1      SUBTLEX_logWF      0.376       -0.189      0.0           0.012   
4       1    SUBTLEX_logW_CD      0.380       -0.211      0.0           0.013   
5       1     SUBTLEX_logCHR      0.380       -0.207      0.0           0.013   
6       1  SUBTLEX_logCHR_CD      0.376       -0.260      0.0           0.016   

   Sample Size  
0          440  
1          440  
2          440  
3          440  
4          440  
5          440  
6          440  


In [9]:
# Univariate linear regression - single-character items - Naming task
import pandas as pd
import statsmodels.formula.api as smf

# Load data
file_path = '27624_expression_7_cleaned.xlsx'  
y_column = 'zRT_Nam_Liu'  # Dependent variable column name
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_Liu","SUBTLEX_logWF","SUBTLEX_logW_CD","SUBTLEX_logCHR","SUBTLEX_logCHR_CD"]  # Predictor variable names
length_column = 'Length'  # Word length column nam

# Read Excel file (select columns)
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Drop rows with missing values in predictors or outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Store regression results for each length
final_results = []

# Iterate over word lengths
lengths = data_cleaned[length_column].unique()  # Get unique lengths
for length in lengths:
    # Filter data for current length
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    # Run univariate regression for each predictor
    for x in x_columns:
        # Fit OLS model
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        # Save metrics
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)  # Add sample size
        })

# Convert results to DataFrame
final_results_df = pd.DataFrame(final_results)

# Print results
print(final_results_df)

   Length           Variable  R-squared  Coefficient  P-value  Standard Error  \
0       1      GPT_FAM_probs      0.325       -0.397      0.0           0.012   
1       1   qwen_FAM_mean_30      0.283       -0.542      0.0           0.018   
2       1      Human_FAM_Liu      0.209       -0.615      0.0           0.025   
3       1      SUBTLEX_logWF      0.289       -0.565      0.0           0.018   
4       1    SUBTLEX_logW_CD      0.290       -0.623      0.0           0.020   
5       1     SUBTLEX_logCHR      0.322       -0.614      0.0           0.019   
6       1  SUBTLEX_logCHR_CD      0.330       -0.772      0.0           0.023   

   Sample Size  
0         2308  
1         2308  
2         2308  
3         2308  
4         2308  
5         2308  
6         2308  


In [ ]:
# Univariate linear regression - multi-character words - LDT task
import pandas as pd
import statsmodels.formula.api as smf

# Load data
file_path = '27624_word_7_cleaned_filtered.xlsx'  
y_column = 'zRT_LDT' 
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_M_Su","SUBTLEX_logWF","SUBTLEX_logW_CD","GPT_FAM_probs_head","SUBTLEX_logCHR_head","SUBTLEX_logCHR_CD_head"]  
length_column = 'Length'  

# Read Excel file
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Clean dataset: keep rows with no missing values in predictors and outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Store regression results per word length
final_results = []

# Group by word length
lengths = data_cleaned[length_column].unique()  
for length in lengths:
    # Filter data for the current length
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    # Run univariate regression for each predictor
    for x in x_columns:
        # Fit OLS model for y ~ x
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        # Save regression metrics
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)  # 添加样本量
        })

# Convert results to DataFrame and display
final_results_df = pd.DataFrame(final_results)

# Print results
print(final_results_df)

    Length                Variable  R-squared  Coefficient  P-value  \
0        2           GPT_FAM_probs      0.377       -0.137   0.0000   
1        2        qwen_FAM_mean_30      0.319       -0.203   0.0000   
2        2          Human_FAM_M_Su      0.297       -0.231   0.0000   
3        2           SUBTLEX_logWF      0.357       -0.196   0.0000   
4        2         SUBTLEX_logW_CD      0.360       -0.212   0.0000   
5        2      GPT_FAM_probs_head      0.061       -0.049   0.0000   
6        2     SUBTLEX_logCHR_head      0.055       -0.076   0.0000   
7        2  SUBTLEX_logCHR_CD_head      0.063       -0.115   0.0000   
8        3           GPT_FAM_probs      0.335       -0.149   0.0000   
9        3        qwen_FAM_mean_30      0.267       -0.205   0.0000   
10       3          Human_FAM_M_Su      0.274       -0.280   0.0000   
11       3           SUBTLEX_logWF      0.176       -0.145   0.0000   
12       3         SUBTLEX_logW_CD      0.183       -0.159   0.0000   
13    

In [10]:
# Univariate linear regression - multi-character words - Naming task -Zhang et al.,2023
import pandas as pd
import statsmodels.formula.api as smf

# Load data
file_path = '27624_word_7_cleaned_filtered_nam.xlsx'  
y_column = 'zRT_Nam_Zhang'   
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_M_Su","SUBTLEX_logWF","SUBTLEX_logW_CD","GPT_FAM_probs_head","SUBTLEX_logCHR_head","SUBTLEX_logCHR_CD_head"]  
length_column = 'Length'  

# Read Excel file
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Ensure rows include no missing values for all predictors and the outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Store regression results
final_results = []

# Iterate over word lengths and run univariate regression for each predictor
lengths = data_cleaned[length_column].unique()  
for length in lengths:
    # Filter data for current length
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    # Run univariate OLS for each predictor
    for x in x_columns:
        # Fit the model y ~ x
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        # Save regression metrics
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)  
        })

# Convert results to DataFrame and print
final_results_df = pd.DataFrame(final_results)

# Print results
print(final_results_df)

    Length                Variable  R-squared  Coefficient  P-value  \
0        2           GPT_FAM_probs      0.116       -0.039   0.0000   
1        2        qwen_FAM_mean_30      0.102       -0.064   0.0000   
2        2          Human_FAM_M_Su      0.041       -0.055   0.0000   
3        2           SUBTLEX_logWF      0.110       -0.048   0.0000   
4        2         SUBTLEX_logW_CD      0.110       -0.053   0.0000   
5        2      GPT_FAM_probs_head      0.097       -0.035   0.0000   
6        2     SUBTLEX_logCHR_head      0.120       -0.061   0.0000   
7        2  SUBTLEX_logCHR_CD_head      0.112       -0.090   0.0000   
8        3           GPT_FAM_probs      0.136       -0.039   0.0317   
9        3        qwen_FAM_mean_30      0.165       -0.067   0.0170   
10       3          Human_FAM_M_Su      0.112       -0.079   0.0535   
11       3           SUBTLEX_logWF      0.051       -0.029   0.2002   
12       3         SUBTLEX_logW_CD      0.070       -0.037   0.1299   
13    

In [24]:
# Univariate linear regression - multi-character words - Naming task
import pandas as pd
import statsmodels.formula.api as smf

# Load data
file_path = '27624_word_7_cleaned.xlsx'  
y_column = 'zRT_Nam_Hendrix'  # zRT_Nam_Hendrix 
x_columns = ["GPT_FAM_probs","qwen_FAM_mean_30","Human_FAM_M_Su","SUBTLEX_logWF","SUBTLEX_logW_CD","GPT_FAM_probs_head","SUBTLEX_logCHR_head","SUBTLEX_logCHR_CD_head"]  
length_column = 'Length'  

# Read Excel file
data = pd.read_excel(file_path, usecols=x_columns + [y_column, length_column])

# Ensure rows include no missing values for all predictors and the outcome
data_cleaned = data.dropna(subset=x_columns + [y_column])

# Store regression results
final_results = []

# Iterate over word lengths and run univariate regression for each predictor
lengths = data_cleaned[length_column].unique()  
for length in lengths:
    # Filter data for current length
    length_data = data_cleaned[data_cleaned[length_column] == length]
    
    # Run univariate OLS for each predictor
    for x in x_columns:
        # Fit the model y ~ x
        formula = f'{y_column} ~ {x}'
        model = smf.ols(formula, data=length_data).fit()
        
        # Save regression metrics
        final_results.append({
            'Length': length,
            'Variable': x,
            'R-squared': round(model.rsquared, 3),
            'Coefficient': round(model.params[x], 3),
            'P-value': round(model.pvalues[x], 4),
            'Standard Error': round(model.bse[x], 3),
            'Sample Size': int(model.nobs)  
        })

# Convert results to DataFrame and print
final_results_df = pd.DataFrame(final_results)

# Print results
print(final_results_df)

   Length                Variable  R-squared  Coefficient  P-value  \
0       2           GPT_FAM_probs      0.193       -0.316      0.0   
1       2        qwen_FAM_mean_30      0.203       -0.528      0.0   
2       2          Human_FAM_M_Su      0.100       -0.384      0.0   
3       2           SUBTLEX_logWF      0.109       -0.349      0.0   
4       2         SUBTLEX_logW_CD      0.111       -0.375      0.0   
5       2      GPT_FAM_probs_head      0.252       -0.336      0.0   
6       2     SUBTLEX_logCHR_head      0.225       -0.507      0.0   
7       2  SUBTLEX_logCHR_CD_head      0.232       -0.762      0.0   

   Standard Error  Sample Size  
0           0.015         1790  
1           0.025         1790  
2           0.027         1790  
3           0.024         1790  
4           0.025         1790  
5           0.014         1790  
6           0.022         1790  
7           0.033         1790  


## Univariate Nonlinear Regression

In [ ]:
# Univariate Nonlinear regression - single-character words - LDT task

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_expression_7_cleaned_filtered.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_LDT'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_Liu',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'SUBTLEX_logCHR',
    'SUBTLEX_logCHR_CD'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")


最终的回归结果表:
 Length          Variable  R_squared  P_value  Sample_Size
      1     GPT_FAM_probs      0.441      0.0          440
      1  qwen_FAM_mean_30      0.404      0.0          440
      1     Human_FAM_Liu      0.269      0.0          440
      1     SUBTLEX_logWF      0.390      0.0          440
      1   SUBTLEX_logW_CD      0.386      0.0          440
      1    SUBTLEX_logCHR      0.404      0.0          440
      1 SUBTLEX_logCHR_CD      0.379      0.0          440


In [13]:
# Univariate Nonlinear regression - single-character words - Naming task

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_expression_7_cleaned.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_Nam_Liu'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_Liu',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'SUBTLEX_logCHR',
    'SUBTLEX_logCHR_CD'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")


最终的回归结果表:
 Length          Variable  R_squared  P_value  Sample_Size
      1     GPT_FAM_probs      0.341      0.0         2308
      1  qwen_FAM_mean_30      0.306      0.0         2308
      1     Human_FAM_Liu      0.210      0.0         2308
      1     SUBTLEX_logWF      0.305      0.0         2308
      1   SUBTLEX_logW_CD      0.295      0.0         2308
      1    SUBTLEX_logCHR      0.337      0.0         2308
      1 SUBTLEX_logCHR_CD      0.334      0.0         2308


In [14]:
# Univariate Nonlinear regression - multi-character words - LDT task

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_word_7_cleaned_filtered.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_LDT'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_M_Su',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'GPT_FAM_probs_head',
    'SUBTLEX_logCHR_head',
    'SUBTLEX_logCHR_CD_head'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")


最终的回归结果表:
 Length               Variable  R_squared  P_value  Sample_Size
      2          GPT_FAM_probs      0.379   0.0000         8386
      2       qwen_FAM_mean_30      0.321   0.0000         8386
      2         Human_FAM_M_Su      0.299   0.0000         8386
      2          SUBTLEX_logWF      0.364   0.0000         8386
      2        SUBTLEX_logW_CD      0.367   0.0000         8386
      2     GPT_FAM_probs_head      0.063   0.0000         8386
      2    SUBTLEX_logCHR_head      0.062   0.0000         8386
      2 SUBTLEX_logCHR_CD_head      0.064   0.0000         8386
      3          GPT_FAM_probs      0.338   0.0000          775
      3       qwen_FAM_mean_30      0.269   0.0000          775
      3         Human_FAM_M_Su      0.275   0.0000          775
      3          SUBTLEX_logWF      0.177   0.0000          775
      3        SUBTLEX_logW_CD      0.185   0.0000          775
      3     GPT_FAM_probs_head      0.017   0.0037          775
      3    SUBTLEX_logCHR_hea

In [16]:
# Univariate Nonlinear regression - multi-character words - Naming task - Zhang et al.,2023

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_word_7_cleaned_filtered_nam.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_Nam_Zhang'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_M_Su',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'GPT_FAM_probs_head',
    'SUBTLEX_logCHR_head',
    'SUBTLEX_logCHR_CD_head'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")


最终的回归结果表:
 Length               Variable  R_squared  P_value  Sample_Size
      2          GPT_FAM_probs      0.120   0.0000         1640
      2       qwen_FAM_mean_30      0.103   0.0000         1640
      2         Human_FAM_M_Su      0.043   0.0000         1640
      2          SUBTLEX_logWF      0.111   0.0000         1640
      2        SUBTLEX_logW_CD      0.112   0.0000         1640
      2     GPT_FAM_probs_head      0.098   0.0000         1640
      2    SUBTLEX_logCHR_head      0.121   0.0000         1640
      2 SUBTLEX_logCHR_CD_head      0.116   0.0000         1640
      3          GPT_FAM_probs      0.166   0.1376           34
      3       qwen_FAM_mean_30      0.252   0.0315           34
      3         Human_FAM_M_Su      0.151   0.1712           34
      3          SUBTLEX_logWF      0.052   0.6495           34
      3        SUBTLEX_logW_CD      0.070   0.5275           34
      3     GPT_FAM_probs_head      0.191   0.0919           34
      3    SUBTLEX_logCHR_hea

In [17]:
# Univariate Nonlinear regression - multi-character words - Naming task - Hendrix et al.,2022

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix

# --------- Step 1: Read data ---------
FILE_PATH = r"D:/0 ECNU/CAILAB/LLM_familiarity/Data/27624_word_7_cleaned.xlsx"  # modify if needed
df = pd.read_excel(FILE_PATH)

# --------- Step 2: Define variables ---------
y_column = 'zRT_Nam_Hendrix'
x_columns = [
    'GPT_FAM_probs',
    'qwen_FAM_mean_30',
    'Human_FAM_M_Su',
    'SUBTLEX_logWF',
    'SUBTLEX_logW_CD',
    'GPT_FAM_probs_head',
    'SUBTLEX_logCHR_head',
    'SUBTLEX_logCHR_CD_head'
]

# Keep rows with complete cases on selected vars (+ Length)
needed_cols = [y_column, 'Length'] + x_columns
df_clean = df.loc[:, needed_cols].copy()

# --------- Step 2.1: Ensure predictors are numeric ---------
# Convert all selected predictors to numeric (coerce non-numeric to NaN)
for col in x_columns + [y_column]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Drop rows with any NaNs in required columns
df_clean = df_clean.dropna(subset=needed_cols).reset_index(drop=True)

# Safety check
non_numeric = [c for c in x_columns if not np.issubdtype(df_clean[c].dtype, np.number)]
if len(non_numeric) > 0:
    raise ValueError(f"Some variables could not be converted to numeric: {', '.join(non_numeric)}")

# --------- Step 3: Unique word lengths ---------
lengths = sorted(df_clean['Length'].unique())

# --------- Prepare final results ---------
final_results = []

# --------- Step 4~8: Loop by Length and predictor; fit OLS with spline ---------
for length_value in lengths:
    length_data = df_clean[df_clean['Length'] == length_value].copy()
    # Skip tiny groups
    if len(length_data) < 5:
        print(f"[Skip] Length={length_value}: sample too small (n={len(length_data)})")
        continue

    for x in x_columns:
        # Build formula: y ~ cr(x, df=4)
        # patsy/statsmodels support cr() via patsy; use smf.ols with formula
        formula = f"{y_column} ~ cr({x}, df=4)"

        try:
            fit = smf.ols(formula, data=length_data).fit()
        except Exception as e:
            print(f"[Error] Length={length_value}, Var={x}: {e}")
            continue

        # --------- Step 8: Collect stats ---------
        final_results.append({
            "Length": length_value,
            "Variable": x,
            "R_squared": round(fit.rsquared, 3),
            "P_value": round(float(fit.f_pvalue), 4),
            "Sample_Size": int(len(length_data))
        })

# --------- Step 9: Print final results table ---------
final_df = pd.DataFrame(final_results)
if not final_df.empty:
    final_df["Variable"] = pd.Categorical(final_df["Variable"], categories=x_columns, ordered=True)
    final_df = final_df.sort_values(by=["Length", "Variable"]).reset_index(drop=True)

    print("\n最终的回归结果表:")
    print(final_df.to_string(index=False))
else:
    print("\nNo models were fitted (empty results).")


最终的回归结果表:
 Length               Variable  R_squared  P_value  Sample_Size
      2          GPT_FAM_probs      0.239      0.0         1790
      2       qwen_FAM_mean_30      0.224      0.0         1790
      2         Human_FAM_M_Su      0.105      0.0         1790
      2          SUBTLEX_logWF      0.110      0.0         1790
      2        SUBTLEX_logW_CD      0.111      0.0         1790
      2     GPT_FAM_probs_head      0.259      0.0         1790
      2    SUBTLEX_logCHR_head      0.238      0.0         1790
      2 SUBTLEX_logCHR_CD_head      0.233      0.0         1790


## Hierarchical Regression

In [ ]:
# Hierarchical regression for LDT (single-character)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_expression_7_cleaned_filtered.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_LDT"
steps = {
    "Step1": ["NoS", "SUBTLEX_logCHR_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA_Liu"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_Liu_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"]                                  # GPT familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

有效样本数: N = 436

===== Step1 =====
R² = 0.430 | ΔR² = 0.430
                            OLS Regression Results                            
Dep. Variable:                zRT_LDT   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     64.94
Date:                Sun, 26 Oct 2025   Prob (F-statistic):           2.04e-50
Time:                        21:43:27   Log-Likelihood:                -496.03
No. Observations:                 436   AIC:                             1004.
Df Residuals:                     430   BIC:                             1029.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [18]:
# Hierarchical regression for Naming (single-character)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_expression_7_cleaned.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_Nam_Liu"
steps = {
    "Step1": ["NoS", "SUBTLEX_logCHR_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA_Liu"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_Liu_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"]                                  # GPT familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

Valid sample size: N = 478

===== Step1 =====
R² = 0.360 | ΔR² = 0.360
                            OLS Regression Results                            
Dep. Variable:            zRT_Nam_Liu   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     53.06
Date:                Mon, 27 Oct 2025   Prob (F-statistic):           1.16e-43
Time:                        14:50:48   Log-Likelihood:                -571.66
No. Observations:                 478   AIC:                             1155.
Df Residuals:                     472   BIC:                             1180.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------

In [ ]:
# # Hierarchical regression for LDT (multi-character)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_word_7_cleaned_filtered.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_LDT"
steps = {
    "Step1": ["Length","NoS", "CF","SUBTLEX_logW_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_M_Su_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"],                                  # GPT familiarity
    "Step5": ["GPT_FAM_probs_head_log"]                                  # GPT head familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

Valid sample size: N = 9546

===== Step1 =====
R² = 0.361 | ΔR² = 0.361
                            OLS Regression Results                            
Dep. Variable:                zRT_LDT   R-squared:                       0.361
Model:                            OLS   Adj. R-squared:                  0.360
Method:                 Least Squares   F-statistic:                     769.2
Date:                Sun, 26 Oct 2025   Prob (F-statistic):               0.00
Time:                        22:16:15   Log-Likelihood:                -11409.
No. Observations:                9546   AIC:                         2.283e+04
Df Residuals:                    9538   BIC:                         2.289e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [20]:
# # Hierarchical regression for Naming (multi-character)(Zhang et al.,2023)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_word_7_cleaned_filtered_nam.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_Nam_Zhang"
steps = {
    "Step1": ["Length","NoS", "CF","SUBTLEX_logW_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_M_Su_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"],                                  # GPT familiarity
    "Step5": ["GPT_FAM_probs_head_log"]                                  # GPT head familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

Valid sample size: N = 1096

===== Step1 =====
R² = 0.161 | ΔR² = 0.161
                            OLS Regression Results                            
Dep. Variable:          zRT_Nam_Zhang   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     29.80
Date:                Mon, 27 Oct 2025   Prob (F-statistic):           8.08e-38
Time:                        14:51:38   Log-Likelihood:                -1459.0
No. Observations:                1096   AIC:                             2934.
Df Residuals:                    1088   BIC:                             2974.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------

In [ ]:
# # Hierarchical regression for Naming (multi-character)(Hendrix et al.,2020)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# ====== 1. Load dataset ======
file_path = "27624_word_7_cleaned.xlsx"
df = pd.read_excel(file_path)

# Define dependent variable and hierarchical predictors
target_col = "zRT_Nam_Hendrix"
steps = {
    "Step1": ["Length","NoS", "CF","SUBTLEX_logW_CD", "NoWF", "NoM", "NoP"],   # Basic lexical features
    "Step2": ["AoA"],                                           # Age of Acquisition
    "Step3": ["Human_FAM_M_Su_log"],                                 # Human familiarity
    "Step4": ["GPT_FAM_probs_log"],                                  # GPT familiarity
    "Step5": ["GPT_FAM_probs_head_log"]                                  # GPT head familiarity
}

# ====== 2. Data cleaning ======
# Keep only the necessary columns
cols_needed = [target_col] + [v for lst in steps.values() for v in lst]
df = df[cols_needed].apply(pd.to_numeric, errors="coerce")
df = df.dropna()
print(f"Valid sample size: N = {len(df)}")

# ====== 3. Standardization (for standardized β coefficients) ======
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

y = df_scaled[target_col]
results = []
prev_r2 = 0

# ====== 4. Hierarchical regression loop ======
for i, (step_name, vars_list) in enumerate(steps.items(), start=1):
    # Include all variables up to the current step
    all_vars = [v for step in list(steps.values())[:i] for v in step]
    X = df_scaled[all_vars]
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    r2 = model.rsquared
    delta_r2 = r2 - prev_r2 if i > 1 else r2
    prev_r2 = r2
    
    print(f"\n===== {step_name} =====")
    print(f"R² = {r2:.3f} | ΔR² = {delta_r2:.3f}")
    print(model.summary())
    
    # Store summary results
    for var in X.columns:
        if var == "const":
            continue
        results.append({
            "Step": step_name,
            "Variable": var,
            "Beta": round(model.params[var], 3),
            "SE": round(model.bse[var], 3),
            "t": round(model.tvalues[var], 3),
            "p": round(model.pvalues[var], 4),
            "R2": round(r2, 3),
            "ΔR2": round(delta_r2, 3)
        })

# ====== 5. Combine and export results ======
res_df = pd.DataFrame(results)
print("\n📊 Hierarchical Regression Summary:")
print(res_df)

Valid sample size: N = 978

===== Step1 =====
R² = 0.315 | ΔR² = 0.315
                            OLS Regression Results                            
Dep. Variable:        zRT_Nam_Hendrix   R-squared:                       0.315
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     74.26
Date:                Sun, 26 Oct 2025   Prob (F-statistic):           2.78e-76
Time:                        22:22:07   Log-Likelihood:                -1203.0
No. Observations:                 978   AIC:                             2420.
Df Residuals:                     971   BIC:                             2454.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------